<h1 style="font-family: times new roman;font-weight: bold;font-size: 40px;">Clustering Documents</h1>

<p style="font-family: times new roman;font-size: 18px;"> Created by: Shashwat Kadam </p>
<hr/>

<p style="font-family: times new roman;font-size: 24px; font-weight:bold">* Dataset description:</p>
<p style="font-family: times new roman;font-size: 18px;">For this mini-project, I am working on a public dataset created in University of California, Irvine (UCI). This is a dataset made especially for studying clustering analysis. The dataset is Bag of Words dataset, in which a vocabulary is present and the counts of the words (term-frequency) in different documents is provided. Following is a more detailed description:</p>

<p style="font-family: times new roman;font-size: 18px;"> The data for this assignment is a bag of words made on collection of KOS Blog entries. There are two different files provided: </p>
<!--<p style="font-family: times new roman;font-size: 18px;color: red; font-weight:bold">TODO: CHANGE THE DATASET FROM ENRON TO KOS / NIPS </p>-->
<ul>
    <li style="font-family: times new roman;font-size: 18px;"> <b>docword.kos.txt</b> </li>
    <li style="font-family: times new roman;font-size: 18px;"> <b>vocab.kos.txt</b></li>
</ul>    

<p style="font-family: times new roman;font-size: 18px; font-weight: bold;"> docword.kos.txt </p>
<p style="font-family: times new roman;font-size: 18px;"> For each text collection, D is the number of documents, W is the
number of words in the vocabulary, and N is the total number of words
in the collection (below, NNZ is the number of nonzero counts in the
bag-of-words). After tokenization and removal of stopwords, the
vocabulary of unique words was truncated by only keeping words that
occurred more than ten times. </p>

<p style="font-family: times new roman;font-size: 18px;"> The file contains metadata of the collection, i.e. values of 'D', 'W' and 'N' as defined above (first 3 lines of the file).This file also contains tuples (docID, wordID, count) which tells that a word having ID <i>wordID</i> is present in document <i>docID</i>, <i>count</i> number of times. Basically it gives <b>term-frequency</b> of term 't' in document 'd' i.e. <i>tf(t, d).</i> Following image will illustrate.</p>

<img src="./images/docword.jpg"/>
<p style="font-family: times new roman;font-size: 12px;">Above image is for <b>docword.enron.txt</b>. Initially I was working on that bag of words but later changed</p>

<p style="font-family: times new roman;font-size: 18px; font-weight: bold;"> vocab.kos.txt </p>
<p style="font-family: times new roman;font-size: 18px;"> This file has lexicographically sorted list of words in the entire collection. These are <b>free from stop-words</b>. The <b>row number is the ID of the word</b>. Following image will illustrate</p>

<img src="./images/vocab.jpg"/>
<p style="font-family: times new roman;font-size: 12px;">Above image is for <b>vocab.enron.txt</b>. Initially I was working on that bag of words but later changed</p>

<p style="font-family: times new roman;font-size: 18px;"> Now that we have understood the data, let's dive in to converting this data into a form suitable for processing. We will also try to pre-process and visualize the data at times, but since the data is already processed and made ready to use, there won't be a need of it as such.</p>

<h1 style="font-family: times new roman; font-weight:bold;">1. Data reading and conversion</h1>

In [3]:
# !pip install tqdm
# Uncomment above line to install 'tqdm' progress bar 

You should consider upgrading via the 'c:\users\shashwat kadam\anaconda3\python.exe -m pip install --upgrade pip' command.


In [5]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from tqdm.auto import trange
from time import time

In [ ]:
import os

DATA_DIR = os.path.join(os.getcwd(), 'data') # data files are present in directory named 'data'

In [ ]:
docword_path = os.path.join(DATA_DIR, 'docword.kos.txt')
vocab_path = os.path.join(DATA_DIR, 'vocab.kos.txt')

In [ ]:
f = open(docword_path, 'r')
file_content = f.read()
f.close()

In [ ]:
raw_data = file_content.split('\n') # separate the line and make a list
print("displaying first 15 lines of content")
print(raw_data[:15])

# Extract the metadata
D, W, N = map(int, raw_data[:3]) # Or map(np.int32, raw_data[:3])

# rest of the data is table
table_data = raw_data[3:-1] 

In [ ]:
print('Making a 2D table:')
table_data = [row.split() for row in tqdm(table_data)] # Making a 2D table (tqdm is progress-bar)

# Create a pandas dataframe
table = pd.DataFrame(table_data, columns=['docID', 'wordID', 'count'], dtype=np.float32)

print("Datatypes: {}".format(table.dtypes))

In [ ]:
table.describe()

In [ ]:
table.head()

In [ ]:
# Getting the vocabulary now
f = open(vocab_path, 'r')
file_content = f.read()
f.close()

In [ ]:
words = file_content.split('\n')
print('few words: {}'.format(words[:10]))
if len(words[-1]) == 0:
    del words[-1]

In [ ]:
# Make a vocabulary dictionary
vocabulary = dict(enumerate(words, 1))
# Display few items from the 'vocabulary'

items = list(vocabulary.items())
print('(wordID, word)')
print(items[:10])

<h1 style="font-family: times new roman; font-weight:bold;">2. Making <i>document vectors</i> and finding <i>Cosine Similarity</i></h1>

<h2 style="font-family: times new roman; font-weight:bold;">2.1. Getting <i>tf-idf</i> scores</h2>

In [ ]:
# Get document frequencied first
document_freqs = table.groupby('wordID')['docID'].count()
document_freqs = pd.DataFrame(document_freqs)
document_freqs.rename(columns={'docID': 'df'}, inplace=True) # now the 'docID' column has accumulated the 'df's
document_freqs

In [ ]:
# Calculate IDF using the formula idf(t) = ln(D/df(t))
dfs = np.array(document_freqs['df'])
if 0 in dfs:
    print(True) # To make sure there aren't any words whose df=0
idfs = np.log(D/dfs)
document_freqs['idf'] = idfs
document_freqs

In [ ]:
# Now that we have idfs of every term, let's compute tf-idf
def get_tf_idf_score(tID, dID, idx):
#     start = time()
    rows = table.loc[table['docID']==dID, ['wordID', 'count']]
    if tID in rows['wordID'].values:
        tf = rows.loc[rows['wordID']==tID, 'count']
    else:
#         end = time()
#         print('Time taken: {} s'.format(end-start))
        return 0.0 # directly return tf_idf = 0.0 because tf = 0.0
    idf = document_freqs.loc[tID]['idf'] if tID in idx else 0.0
    tf_idf = tf * idf
#     end = time()
#     print('Time taken: {} s'.format(end-start))
    return float(tf_idf)
idx = document_freqs.index
print(get_tf_idf_score(61, 1, idx))

<h2 style="font-family: times new roman; font-weight:bold;">2.2. Making normalized document vectors</h2>

In [ ]:
def get_valid_word_IDs(dID):
    # returns list of wordID present in document having ID 'dID'
    return np.array(table.loc[table['docID'] == dID, 'wordID'], dtype='int')


In [ ]:
def get_document_vector(dID):
    # Returns a numpy array which is the normalized document vector of document 'dID'.
#     start = time()
    trunc_doc_vector = get_valid_word_IDs(dID)
    tf_idfs = [get_tf_idf_score(tID, dID, idx) for tID in trunc_doc_vector]
    d1 = np.zeros((W+1,))
    d1[trunc_doc_vector] = 1.0
    d1[d1 == 1.0] = tf_idfs
    # Normalize the vector
    norm = np.linalg.norm(d1)
    if norm == 0:
        return d1
    
#     end = time()
#     print('Time elapsed: {} s'.format(end-start))
    return d1 / norm

In [ ]:

doc_vecs = []
for dID in tqdm(range(1, D+1), desc="Progress", ncols="980"):
    doc_vecs.append(get_document_vector(dID))

<h3 style="font-family: times new roman; font-weight:bold;">2.2.1. Saving the vectors on a file to avoid re-computation</h3>

In [ ]:
OUTPUT_PATH = os.path.join(os.getcwd(), "precomputed_data")

if os.path.exists(OUTPUT_PATH) == False:
    print("Creating directory {}...".format(OUTPUT_PATH))
    os.mkdir(OUTPUT_PATH)
    print("Directory {} created!".format(OUTPUT_PATH))
    
FILE_NAME = "kos_document_vectors.csv"

data_block = np.array(doc_vecs)
data_block = pd.DataFrame(data_block)
data_block.to_csv(os.path.join(OUTPUT_PATH, FILE_NAME))

<h2 style="font-family: times new roman; font-weight:bold;">2.3. Calculating <i>Cosine similarites</i></h2>

In [ ]:
# Now that the document vectors are normalized, simple dot product will give the value of 'cos' of angle between two documents
doc_vecs = pd.read_csv(os.path.join(OUTPUT_PATH, FILE_NAME))

In [ ]:
doc_vecs

In [ ]:
doc_vecs1 = np.array(doc_vecs)

In [ ]:
doc_vecs = doc_vecs1[:, 1:]

In [ ]:
rows, cols = doc_vecs.shape
print('No. of documents: {} No. of terms: {}'.format(rows, cols))

similarity_matrix = np.zeros(shape=(rows, rows))
print(similarity_matrix.shape)
for i in trange(rows, ncols='980'):
    for j in range(rows):
        similarity_matrix[i, j] = np.dot(doc_vecs[i], doc_vecs[j])

In [ ]:
print(similarity_matrix)

In [ ]:
print("Rounding off to 7 decimal places")
similarity_matrix = np.around(similarity_matrix, decimals=7)
print(similarity_matrix)

In [ ]:
# Save similarity matrix to a CSV file to avoid recomputation
file_data = pd.DataFrame(similarity_matrix)
if os.path.exists(OUTPUT_PATH):
    print("directory exists. Saving the file...")
    file_data.to_csv(os.path.join(OUTPUT_PATH, "kos_similarity_matrix.csv"))
else:
    print("Creating directory {}. And saving the file...".format(OUTPUT_PATH))
    os.mkdir(OUTPUT_PATH)
    file_data.to_csv(os.path.join(OUTPUT_PATH, "kos_similarity_matrix.csv"))
print("File saved. Check the directory {} to verify...".format(OUTPUT_PATH))    

In [ ]:
difference_matrix = 1. - similarity_matrix
print("Difference matrix:")
print(difference_matrix)

We now have both 'difference' and 'similarity' matrices

<h1 style="font-family: times new roman; font-weight:bold;">3. Clustering</h1>
<p style="font-family: times new roman; font-size:18px;"> Now we have calculated and saved document vectors and similarity matrix. If you want to recompute the vectors and matrix, run the code from <i>Section 2.</i> Otherwise just load the data from saved files. It will be much easier. If you want to save vectors and similarity matrix from other dataset, simply change the file paths accordingly, and re-run from <i>Section 2.</i> onwards</p>

<h2 style="font-family: times new roman; font-weight:bold;">3.1. Agglomerative Clustering (Hierarchical clustering)</h2>

In [ ]:
from sklearn.cluster import AgglomerativeClustering

model = AgglomerativeClustering(affinity='precomputed', n_clusters=5, linkage='complete')
model.fit(difference_matrix)
# print(list(model.labels_))
print(len(model.labels_))

In [ ]:
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram

def plot_dendrogram(model, **kwargs):

    # Children of hierarchical clustering
    children = model.children_

    # Distances between each pair of children
    # Since we don't have this information, we can use a uniform one for plotting
    distance = np.arange(children.shape[0])

    # The number of observations contained in each cluster level
    no_of_observations = np.arange(2, children.shape[0]+2)

    # Create linkage matrix and then plot the dendrogram
    linkage_matrix = np.column_stack([children, distance, no_of_observations]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

In [ ]:
figure = plt.figure(figsize=(100,110))
plt.title("Dendrogram")
plot_dendrogram(model, labels=model.labels_)
plt.show()

In [ ]:
model.labels_

In [ ]:
def get_clusters_from_labels(labels, n_clusters=5):
    cluster_bag = {i+1:[] for i in range(n_clusters)}
    for i in range(len(labels)):
        cluster_bag[labels[i]+1].append(i+1)
    return cluster_bag    
    

In [ ]:
cb = get_clusters_from_labels(model.labels_, 5)

In [ ]:
for k, v in cb.items():
    print("Cluster {} contains documents: {}]\nTotal {} documents".format(k, v, len(v)))
    print('*'*30)

<h2 style="font-family: times new roman; font-weight:bold;">3.2. K-Means Clustering</h2>

In [ ]:
from sklearn.cluster import KMeans
k_means = KMeans(init='k-means++', n_clusters=8, n_init=12)

In [ ]:
doc_vecs = pd.read_csv("./precomputed_data/kos_document_vectors.csv")
dvs = np.array(doc_vecs)
dvs = dvs[:, 1:]

In [ ]:
k_means.fit(dvs)

In [ ]:
k_means.labels_

In [ ]:
cluster_bags = get_clusters_from_labels(k_means.labels_, 8)

In [ ]:
for k, v in cluster_bags.items():
    print("Cluster {} contains documents: {}]\nTotal {} documents".format(k, v, len(v)))
    print('*'*30)

<h2 style="font-family: times new roman; font-weight:bold;">3.3. DBSCAN Clustering</h2>

In [6]:
from sklearn.cluster import DBSCAN

sim_mat = pd.read_csv("./precomputed_data/kos_similarity_matrix.csv")
sim_mat = np.array(sim_mat)
sim_mat = sim_mat[:, 1:]
diff_mat = 1 - sim_mat
dbscan = DBSCAN(eps=1.0, min_samples=2)
dbscan.fit(diff_mat)

DBSCAN(algorithm='auto', eps=1.0, leaf_size=30, metric='euclidean',
    metric_params=None, min_samples=2, n_jobs=None, p=None)